In [2]:
#!/usr/bin/env python

# Copyright (c) 2019 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import random
import time

In [3]:
actor_list = []

# In this tutorial script, we are going to add a vehicle to the simulation
# and let it drive in autopilot. We will also create a camera attached to
# that vehicle, and save all the images generated by the camera to disk.

In [4]:
# 시뮬레이터에 요청을 보낼 클라이언트를 생성. 
# 여기서는 시뮬레이터가 포트 2000의 로컬 호스트에서 요청을 수신 가정.

client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [5]:
# 클라이언트가 생기면 현재 실행 중인 월드를 검색할 수 있다.
world = client.get_world()

In [6]:
# # pedestrian code

# world.set_pedestrians_seed(1235)
# ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))

# ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
# ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=180.0, roll=0.0)
# start_trans = carla.Transform(ped_start_location, ped_start_rotation)

# ped_end_location = carla.Location(x=-60.3, y=2.7, z=0.2)
# ped_end_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
# dst_trans = carla.Transform(ped_end_location, ped_end_rotation)

# ped = world.spawn_actor(ped_bp, start_trans)
# walker_controller_bp = world.get_blueprint_library().find('controller.ai.walker')
# controller = world.spawn_actor(walker_controller_bp, carla.Transform(), ped)
# controller.start()
# controller.set_max_speed(2)

# # Set initial target location for the pedestrian
# # target_location = ped_end_location

# # traffic light
# traffic_light = world.get_actor(16)
# current_traffic_light = carla.TrafficLightState.Green

# # adding an actor to an actor list
# actor_list.append(ped)
# actor_list.append(controller)

# while True:
#     # current_location = ped.get_location()
#     # print(current_location)
#     x = ped.get_location().x
#     y = ped.get_location().y        # restrict Unexpected movement

#     # if traffic_light.get_state() == current_traffic_light:
#     #     pass
#     # else:
#     #     current_traffic_light = traffic_light.get_state()
#     #     if current_traffic_light == carla.TrafficLightState.Red:
#     #         controller.set_max_speed(0)
#     #         controller.stop()
#     #     elif current_traffic_light == carla.TrafficLightState.Green:
#     #         controller.set_max_speed(2)
#     #         controller.start()
    
#     print(traffic_light.get_state())

#     # if current_location.distance(ped_start_location) < 1.2:
#     #     # If the pedestrian has reached the range start, set the target location to the range end
#     #     # target_location = ped_end_location
#     #     controller.go_to_location(ped_end_location)
        
#     # elif current_location.distance(ped_end_location) < 1.2:
#     #     # If the pedestrian has reached the range end, set the target location to the range start
#     #     # target_location = ped_start_location
#     #     controller.go_to_location(ped_start_location)
    
#     if x < -62.0 or y < 0.5:
#         controller.go_to_location(ped_start_location)
#         # target_location = ped_start_location
#     elif x > -32.0 or y < 0.5:
#         controller.go_to_location(ped_end_location)
#         # target_location = ped_end_location

#     # controller.go_to_location(target_location)

#     world.tick()


In [11]:
# pedestrian code without ai 

world.set_pedestrians_seed(1235)
# ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
ped_bp = world.get_blueprint_library().find("walker.pedestrian.0002")

ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=180.0, roll=0.0)
start_trans = carla.Transform(ped_start_location, ped_start_rotation)

ped_end_location = carla.Location(x=-60.3, y=2.7, z=0.2)
ped_end_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
dst_trans = carla.Transform(ped_end_location, ped_end_rotation)

ped = world.spawn_actor(ped_bp, start_trans)
ped_controller = carla.WalkerControl()
ped_controller.speed = 0.2
ped_controller.direction = carla.Vector3D(ped_end_location.x - ped_start_location.x, ped_end_location.y - ped_start_location.y, 0)
ped_controller.jump = False

# adding an actor to an actor list
actor_list.append(ped)

def move_pedestrian(ped, ped_controller, ped_start_location, ped_end_location):
    # Define the section where the pedestrian will walk
    section_start = -54.3
    section_end = -39.3
    
    # Move towards the end point
    ped.apply_control(ped_controller)
    distance_to_end = ped.get_location().distance(ped_end_location)
    if distance_to_end < 1.0:
        # Change direction and increase speed if at the end point
        ped_controller.direction = carla.Vector3D(ped_start_location.x - ped_end_location.x, ped_start_location.y - ped_end_location.y, 0)
        # ped_controller.speed = 0.25 if ped.get_location().x < section_start else 0.1
        # ped.apply_control(ped_controller)
    
    # Move towards the start point
    ped.apply_control(ped_controller)
    distance_to_start = ped.get_location().distance(ped_start_location)
    if distance_to_start < 1.0:
        # Change direction and increase speed if at the start point
        ped_controller.direction = carla.Vector3D(ped_end_location.x - ped_start_location.x, ped_end_location.y - ped_start_location.y, 0)
        # ped_controller.speed = 0.25 if ped.get_location().x > section_end else 0.1
        # ped.apply_control(ped_controller)

while True:
    # # move towards the end point
    # ped.apply_control(ped_controller)
    # while ped.get_location().distance(ped_end_location) > 1.0:
    #     if ped.get_location().x < -54.3:
    #         ped_controller.speed = 0.25
    #         ped.apply_control(ped_controller)
    #     pass
    
    # # change the direction of the walker control to move towards the start point
    # ped_controller.direction = carla.Vector3D(ped_start_location.x - ped_end_location.x, ped_start_location.y - ped_end_location.y, 0)
    # ped_controller.speed = 0.1
    
    # # move towards the start point
    # ped.apply_control(ped_controller)
    # while ped.get_location().distance(ped_start_location) > 1.0:
    #     if ped.get_location().x > -39.3:
    #         ped_controller.speed = 0.25
    #         ped.apply_control(ped_controller)
    #     pass
    
    # # change the direction of the walker control to move towards the end point
    # ped_controller.direction = carla.Vector3D(ped_end_location.x - ped_start_location.x, ped_end_location.y - ped_start_location.y, 0)
    # ped_controller.speed = 0.1

    move_pedestrian(ped, ped_controller, ped_start_location, ped_end_location)

KeyboardInterrupt: 

In [12]:
ped.destroy()

True

In [96]:
# pedestrian code without ai , add traffic

world.set_pedestrians_seed(1235)
# ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
ped_bp = world.get_blueprint_library().find("walker.pedestrian.0002")

ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=180.0, roll=0.0)
start_trans = carla.Transform(ped_start_location, ped_start_rotation)

ped_end_location = carla.Location(x=-60.3, y=2.7, z=0.2)
ped_end_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
dst_trans = carla.Transform(ped_end_location, ped_end_rotation)

ped = world.spawn_actor(ped_bp, start_trans)
ped_controller = carla.WalkerControl()
ped_controller.speed = 0.1
ped_controller.direction = carla.Vector3D(ped_end_location.x - ped_start_location.x, ped_end_location.y - ped_start_location.y, 0)
ped_controller.jump = False

# traffic light
traffic_light = world.get_actor(16)

# adding an actor to an actor list
actor_list.append(ped)

def check_traffic(traffic_light, controller, pedestrian):
    global current_traffic_light
    if traffic_light.get_state() == current_traffic_light:
        pass
    elif traffic_light.get_state() == carla.TrafficLightState.Yellow:
        pass
    else:
        current_traffic_light = traffic_light.get_state()
        if current_traffic_light == carla.TrafficLightState.Red:
            controller.speed = 0
            pedestrian.apply_control(controller)
            print('speed 0')
        elif current_traffic_light == carla.TrafficLightState.Green:
            controller.speed = 0.25
            pedestrian.apply_control(controller)
            print('speed 0.25')
        current_traffic_light = traffic_light.get_state()
    
    # print(traffic_light.get_state())
    

while True:
    if traffic_light.get_state() == carla.TrafficLightState.Red:
        ped_controller.speed = 0.0
        while traffic_light.get_state() == carla.TrafficLightState.Red:
            time.sleep(0.1)
    elif traffic_light.get_state() == carla.TrafficLightState.Green: 
        ped_controller.speed = 1.0
    # move towards the end point
    ped.apply_control(ped_controller)
    while ped.get_location().distance(ped_end_location) > 1.0:
        # Check if the traffic light is red when the pedestrian is at the sidewalk block
        if ped.get_location().x < -54.3 and traffic_light.get_state() == carla.TrafficLightState.Red:
            ped_controller.speed = 0.0
            while traffic_light.get_state() == carla.TrafficLightState.Red:
                time.sleep(0.1)
            ped_controller.speed = 0.1
        elif ped.get_location().x < -54.3 and traffic_light.get_state() == carla.TrafficLightState.Green:
            ped_controller.speed = 0.25
        else:
            ped_controller.speed = 0.1
        ped.apply_control(ped_controller)
    
    # change the direction of the walker control to move towards the start point
    ped_controller.direction = carla.Vector3D(ped_start_location.x - ped_end_location.x, ped_start_location.y - ped_end_location.y, 0)
    ped_controller.speed = 0.1
    
    # move towards the start point
    ped.apply_control(ped_controller)
    while ped.get_location().distance(ped_start_location) > 1.0:
        if ped.get_location().x > -39.3 and traffic_light.get_state() == carla.TrafficLightState.Red:
            ped_controller.speed = 0.0
            while traffic_light.get_state() == carla.TrafficLightState.Red:
                time.sleep(0.1)
            ped_controller.speed = 0.1
        elif ped.get_location.x < -39.3 and traffic_light.get_state() == carla.TrafficLightState.Green:
            ped_controller.speed = 0.25
        else:   
            ped_controller.speed = 0.1
        ped.apply_control(ped_controller)
    
    # change the direction of the walker control to move towards the end point
    ped_controller.direction = carla.Vector3D(ped_end_location.x - ped_start_location.x, ped_end_location.y - ped_start_location.y, 0)
    ped_controller.speed = 0.1

AttributeError: 'Boost.Python.function' object has no attribute 'x'

In [45]:
world.set_pedestrians_seed(1235)
# ped_bp = random.choice(world.get_blueprint_library().filter("walker.pedestrian.*"))
ped_bp = world.get_blueprint_library().find("walker.pedestrian.0002")

ped_start_location = carla.Location(x=-34.3, y=2.7, z=0.2)
ped_start_rotation = carla.Rotation(pitch=-0.3, yaw=180.0, roll=0.0)
start_trans = carla.Transform(ped_start_location, ped_start_rotation)

ped_end_location = carla.Location(x=-60.3, y=2.7, z=0.2)
ped_end_rotation = carla.Rotation(pitch=-0.3, yaw=-180.0, roll=0.0)
dst_trans = carla.Transform(ped_end_location, ped_end_rotation)

ped = world.spawn_actor(ped_bp, start_trans)

In [47]:
ped_start_location.x += 5
ped.set_location(ped_start_location)


In [48]:
print(ped.get_location())

Location(x=-39.299999, y=2.700000, z=0.951500)


In [41]:
traffic_light = world.get_actors().filter('traffic.*')
print(traffic_light)

[Actor(id=23, type=traffic.traffic_light), Actor(id=22, type=traffic.traffic_light), Actor(id=21, type=traffic.traffic_light), Actor(id=20, type=traffic.traffic_light), Actor(id=19, type=traffic.traffic_light), Actor(id=18, type=traffic.traffic_light), Actor(id=17, type=traffic.traffic_light), Actor(id=16, type=traffic.traffic_light), Actor(id=15, type=traffic.traffic_light), Actor(id=14, type=traffic.traffic_light), Actor(id=13, type=traffic.traffic_light), Actor(id=12, type=traffic.traffic_light), Actor(id=11, type=traffic.traffic_light), Actor(id=10, type=traffic.traffic_light), Actor(id=9, type=traffic.traffic_light), Actor(id=8, type=traffic.unknown), Actor(id=7, type=traffic.unknown), Actor(id=6, type=traffic.unknown), Actor(id=5, type=traffic.yield), Actor(id=4, type=traffic.stop), Actor(id=3, type=traffic.stop), Actor(id=2, type=traffic.stop)]


In [1]:
# client.apply_batch([carla.command.DestroyActor(x) for x in actor_list])

ped.destroy()
# controller.destroy()

NameError: name 'ped' is not defined

In [10]:
asdf = world.get_actors().filter('walker.*')
asdf[0].destroy()

True